In [ ]:
import sys
sys.path.append('../src')
import numpy as np
import os
from evaluation import load_dataset, load_results, convert_results, compute_scores

In [ ]:
def print_scores(mode, num_objects='2_3', filename_base='result_{}.h5', num_tests=5):
    for name_data_base in name_data_base_list:
        is_ordered = False if mode == 'sep' else name_data_base in ['rgb_1', 'rgb_3']
        for name_data_sub in name_data_sub_list:
            print('{}_{}'.format(name_data_base, name_data_sub))
            images, labels_ami, labels_mse = load_dataset(
                os.path.join(folder_data_base, '{}_{}'.format(mode, name_data_base)),
                '{}_{}'.format(name_data_sub, num_objects))
            folder_result = '{}_{}_{}'.format(mode, name_data_base, name_data_sub)
            scores_list = {key: [] for key in ['AMI', 'MSE', 'OCA', 'OOA_ORI', 'OOA_ADJ']}
            for model_id in range(num_tests):
                results = load_results(folder_result, filename_base.format(model_id))
                gamma, pres, coef, recon_scene, recon_objects = convert_results(results)
                ami_score, mse_score, oca_score, ooa_score_ori, ooa_score_adj = compute_scores(
                    images, labels_ami, labels_mse, gamma, pres, coef, recon_objects, is_ordered)
                scores_list['AMI'].append(ami_score * 1e2)
                scores_list['MSE'].append(mse_score * 1e3)
                scores_list['OCA'].append(oca_score * 1e2)
                scores_list['OOA_ORI'].append(ooa_score_ori * 1e2)
                scores_list['OOA_ADJ'].append(ooa_score_adj * 1e2)
            scores_mean = {key: np.mean(val) for key, val in scores_list.items()}
            scores_std = {key: np.std(val) for key, val in scores_list.items()}
            print('AMI(%):{:5.3g} {:.0e}'.format(scores_mean['AMI'], scores_std['AMI'] * 2), end=' ' * 4)
            print('MSE(e-3):{:4.1f} {:.0e}'.format(scores_mean['MSE'], scores_std['MSE'] * 2), end=' ' * 4)
            print('OCA(%):{:5.3g} {:.0e}'.format(scores_mean['OCA'], scores_std['OCA'] * 2), end=' ' * 4)
            if is_ordered:
                print('OOA_ORI(%):{:5.3g} {:.0e}'.format(scores_mean['OOA_ORI'], scores_std['OOA_ORI'] * 2), end=' ' * 4)
                print('OOA_ADJ(%):{:5.3g} {:.0e}'.format(scores_mean['OOA_ADJ'], scores_std['OOA_ADJ'] * 2), end=' ' * 4)
            print()

folder_data_base = '../data'
name_data_base_list = ['gray', 'rgb_1', 'rgb_2', 'rgb_3', 'rgb_4']
name_data_sub_list = ['shapes', 'mnist']

In [ ]:
# Evaluate on images containing 2 or 3 objects (without occlusion)
print_scores('sep')

In [ ]:
# Evaluate on images containing 2 or 3 objects (with occlusion)
print_scores('occ')

In [ ]:
# Evaluate on images containing 4 objects (without occlusion, K = 4)
print_scores('sep', num_objects='4', filename_base='general_4_result_{}.h5')

In [ ]:
# Evaluate on images containing 4 objects (without occlusion, K = 10)
print_scores('sep', num_objects='4', filename_base='general_10_result_{}.h5')

In [ ]:
# Evaluate on images containing 4 objects (with occlusion, K = 4)
print_scores('occ', num_objects='4', filename_base='general_4_result_{}.h5')

In [ ]:
# Evaluate on images containing 4 objects (with occlusion, K = 10)
print_scores('occ', num_objects='4', filename_base='general_10_result_{}.h5')